In [2]:
#Part 1: Data Preprocessing [7 marks]
#Download the Term Marketing - Banking data set from the following link: https://www.kaggle.com/datasets/prakharrathi25/banking-dataset-marketing-targetsLinks to an external site.
#Clean and explore the data. Summarize your findings.
#Build a pipeline to process the data (i.e., One Hot Encoding, Scaling, and any other relevant transformations)
#Explain your pre-processing approach and justify the transformations you have made.
#Part 2: Build Baseline Models [3 marks]
#Create baseline models using logistic regression and k-NN. Tune the models.
#Evaluate these models using the performance metrics you deem appropriate.
#Part 3: Ensemble Modelling [6 marks]
#Random Forest

#Build a Random Forest model.
#Tune hyperparameters.
#AdaBoost

#Build an AdaBoost model.
#Tune hyperparameters.
#Bagging Classifier

#Build a Bagging Classifier with a base estimator of your choice.
#Tune hyperparameters.
#Voting Classifier

#Build a Voting Classifier using a mix of at least three different classification models.
#Experiment with both hard and soft voting.
#Part 4: Performance Comparison [4 marks]
#Compare the performance of all models (including the baseline models). Consider both the time required for the models to run, and the performance of the models on the data set.
#Part 5: Interpretation and Justification [8 marks]
#Why did the ensemble models perform the way they did?

#Beyond the hyperparameters, use your understanding of how the models work to explain why you think the models performed they way they did on the given data set. Was the result what you were expecting? Why or why not?
#If you had to pick one model to implement in business process, which would it be and why?

#Discuss the business implications.
#Consider not only performance metrics but also computational cost and interpretability.
#What decision criteria did you use to arrive at this conclusion?

#Precision-Recall trade-off? Computational cost? Others?
#Grammar and Spelling [2 marks]

# Import Libraries

In [1]:
import pandas as pd

# Part 1: Data Preprocessing

In [5]:
# Load the Dataset - requires delimeter parameter (data is divided with semi-colons, not commas)
df = pd.read_csv('https://raw.githubusercontent.com/EthanRosehart/DS2_Assignments/main/train.csv', delimiter=';')

In [6]:
# Check the data
df.head(10)

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no
5,35,management,married,tertiary,no,231,yes,no,unknown,5,may,139,1,-1,0,unknown,no
6,28,management,single,tertiary,no,447,yes,yes,unknown,5,may,217,1,-1,0,unknown,no
7,42,entrepreneur,divorced,tertiary,yes,2,yes,no,unknown,5,may,380,1,-1,0,unknown,no
8,58,retired,married,primary,no,121,yes,no,unknown,5,may,50,1,-1,0,unknown,no
9,43,technician,single,secondary,no,593,yes,no,unknown,5,may,55,1,-1,0,unknown,no


Check for Missing Values & Data Types

In [ ]:
# Check for missing values and data types
df.info()

# Display the percentage of missing values (if any)
missing_values = df.isnull().sum() / len(df) * 100
print("Missing Values:\n", missing_values[missing_values > 0])
